In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import igraph
import time
import gc
from os import path
import ipython_memory_usage
import numpy as np
from scipy.sparse import csr_matrix, lil_matrix

In [2]:
%ipython_memory_usage_start

'memory profile enabled'

In [2] used 0.1094 MiB RAM in 0.15s, peaked 0.00 MiB above current, total RAM usage 108.02 MiB


In [3]:
vertices = pd.read_csv('data\\vertices.csv') ## вершины ребер, то есть юр.лица
edges = pd.read_csv('data\\edges.csv') ## ребра графа, то есть транзакции между юр.лицами
test = pd.read_csv('data\\ids.csv')#.drop('rec_num_of_edges', axis = 'columns')

In [3] used 190.8789 MiB RAM in 2.10s, peaked 81.16 MiB above current, total RAM usage 298.90 MiB


In [4]:
rec_num_of_edges = test.merge(vertices, how = 'outer')[['id', 'rec_num_of_edges']]
rec_num_of_edges = rec_num_of_edges.fillna(np.inf)
rec_num_of_edges['cur_num_of_edges'] = 0
rec_num_of_edges['id'] = rec_num_of_edges['id'].astype(int)

In [4] used 42.0820 MiB RAM in 0.65s, peaked 108.52 MiB above current, total RAM usage 340.98 MiB


In [5]:
test = test.drop('rec_num_of_edges', axis = 'columns')

In [5] used 0.0273 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 341.01 MiB


In [6]:
my_encoder = LabelEncoder()
vertices['company_type']= my_encoder.fit_transform(vertices['company_type'])

In [6] used -5.0781 MiB RAM in 0.33s, peaked 16.75 MiB above current, total RAM usage 335.93 MiB


In [7]:
vertices['is_true'] = 0
vert_id_istrue = vertices[['id', 'is_true']]
vert_no_true = vertices.drop(['is_true'], axis = 'columns')

In [7] used 76.1484 MiB RAM in 0.23s, peaked 0.00 MiB above current, total RAM usage 412.08 MiB


In [17]:
from lightgbm import LGBMClassifier
import catboost as cat
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

In [17] used 2.3906 MiB RAM in 0.13s, peaked 0.00 MiB above current, total RAM usage 2939.48 MiB


In [9]:
def selecting_top100(predicted_prob):
    predicted_edges = []
    
    for edge in predicted_prob:
        if (edge not in predicted_edges) &\
        (tuple([edge[place] for place in [1,0]]) not in predicted_edges) &\
        (len(predicted_edges) <= 99_999):
            predicted_edges.append(edge)
            
        elif (len(predicted_edges) > 99_999):
            break
    
    predicted_edges = pd.DataFrame(predicted_edges, columns = ['id_1', 'id_2'])
    return(predicted_edges)

In [9] used 0.0234 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 427.29 MiB


In [10]:
def selecting_top100_limited(predicted_prob):
    global rec_num_of_edges, edge
    predicted_edges = []
    
    for edge in predicted_prob:
        if (edge not in predicted_edges) &\
        (tuple([edge[place] for place in [1,0]]) not in predicted_edges) &\
        (len(predicted_edges)<= 99_999) &\
        (rec_num_of_edges[rec_num_of_edges['id'] == edge[0]]['rec_num_of_edges'] > rec_num_of_edges[rec_num_of_edges['id'] == edge[0]]['cur_num_of_edges']).values[0] &\
        (rec_num_of_edges[rec_num_of_edges['id'] == edge[1]]['rec_num_of_edges'] > rec_num_of_edges[rec_num_of_edges['id'] == edge[1]]['cur_num_of_edges']).values[0]:
            predicted_edges.append(edge)
            rec_num_of_edges['cur_num_of_edges'] = np.where(rec_num_of_edges['id'] == edge[0], 
                                                            rec_num_of_edges['cur_num_of_edges'] + 1, 
                                                            rec_num_of_edges['cur_num_of_edges'])
            rec_num_of_edges['cur_num_of_edges'] = np.where(rec_num_of_edges['id'] == edge[1], 
                                                            rec_num_of_edges['cur_num_of_edges'] + 1, 
                                                            rec_num_of_edges['cur_num_of_edges'])
        elif (len(predicted_edges) > 99_999):
            break
            
    predicted_edges = pd.DataFrame(predicted_edges, columns = ['id_1', 'id_2'])
    return(predicted_edges)

In [10] used 0.1992 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 427.49 MiB


In [48]:
def predicting(model, model_name, cat_features = True):
    global X, predicted_edges, firm, temp, predicted_prob
    predicted_prob = pd.DataFrame()
    
    if type(model).__name__ == 'CatBoostClassifier':
        print('Check catfeatures')
    
    for firm in test['id']:
        start = time.time()

        temp_1 = edges[edges['id_1'] == firm]['id_2'].rename('id')
        temp_2 = edges[edges['id_2'] == firm]['id_1'].rename('id')
        temp = pd.DataFrame(temp_1.append(temp_2, ignore_index = True))
        temp['is_true'] = 1
        temp = temp.drop_duplicates()

        temp = temp.append(vert_id_istrue, ignore_index = True)
        temp = temp.drop_duplicates(subset = ['id'])
        temp = temp.merge(vert_no_true, on = ['id'], how = 'inner')

        X = temp.drop(['is_true', 'id'], axis = 'columns')
        
        if cat_features == True:
            X['main_okved'] = X['main_okved'].astype(str)
            
        y = temp['is_true']
        model_trained = model.fit(X, y)
        proba = pd.DataFrame(model_trained.predict_proba(X))

        proba['id_1'] = firm
        proba['id_2'] = temp['id'].values
        proba['is_true'] = y.values
        predicted_prob = predicted_prob.append(proba)

        del temp_1, temp_2, temp, X, y
        gc.collect()
        
    predicted_prob = predicted_prob[predicted_prob['is_true'] != 1]
    
    ## Сортируем вершины по вероятности правдивости
    )
    predicted_prob = predicted_prob.sort_values([1], ascending = False)
    
    predicted_prob = predicted_prob.sort_values([1], ascending = False)[['id_1', 'id_2']]

    predicted_prob = [edge[1:] for edge in predicted_prob.iloc[:10_000_000,:].to_records().tolist()]
    
    predicted_edges = selecting_top100(predicted_prob =  predicted_prob)

    predicted_edges.to_csv('{}.csv'.format(model_name), index = False)

In [48] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 1196.82 MiB


In [53]:
display(X)

,main_okved,region_code,company_type
0,49.32,77,3
1,63.99,66,3
2,49.32,77,2
3,49.32,32,3
4,62.09,23,2
...,...,...,...
1534744,63.99,77,2
1534745,47.19,66,2
1534746,41.20,77,2
1534747,74.20,33,2


In [53] used 0.0000 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 525.04 MiB


In [ ]:
model = VotingClassifier(
    estimators = [('0', CatBoostClassifier(n_estimators = 100, random_state = 42, verbose = False)),
                  ('1', CatBoostClassifier(n_estimators = 100, random_state = 322, verbose = False)),
                  ('2', CatBoostClassifier(n_estimators = 100, random_state = 3443506946, verbose = False)),
                  ('3', CatBoostClassifier(n_estimators = 100, random_state = 69, verbose = False)),
                  ('4', CatBoostClassifier(n_estimators = 100, random_state = 1, verbose = False)),
                  ('5', CatBoostClassifier(n_estimators = 100, random_state = 22, verbose = False)),
        
    ],
    voting = 'soft',
)
model_name = 'VotingClassifier Catboost. n_estimators = 100. CPU. Full graph. As in baseline'
predicting(model,  model_name, cat_features = False)